In [ ]:
#@title Импорты
import pandas
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelEncoder



In [ ]:
#@title #1. Скачайте датасет ссылка #https://www.kaggle.com/datasets/gauravduttakiit/loan-defaulter #Цель - предсказать, является ли заявитель неплательщиком (выплатит кредит банку или нет) #TARGET действует как зависимая переменная - является ли заявитель неплательщиком или нет
#Application data: 122 features | Previous application data: 37 features

application_data = pd.read_csv('/content/application_data.csv')
previous_application = pd.read_csv('/content/previous_application.csv')

application_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


In [ ]:
#@title #2. Сделайте предобработку данных #Удаление лишних и бесполезных данных #Заполнение пропущенных значений #Выбросы (средн. знач.)

application_data.drop(labels=['OWN_CAR_AGE', 'EXT_SOURCE_1', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'TOTALAREA_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'],axis=1,inplace=True)

application_data.drop(labels=['FLAG_MOBIL',  'FLAG_EMP_PHONE',  'FLAG_WORK_PHONE',  'FLAG_CONT_MOBILE',  'FLAG_PHONE',  'FLAG_EMAIL',  'FLAG_DOCUMENT_2',  'FLAG_DOCUMENT_3',  'FLAG_DOCUMENT_4',  'FLAG_DOCUMENT_5',  'FLAG_DOCUMENT_6',  'FLAG_DOCUMENT_7',  'FLAG_DOCUMENT_8',  'FLAG_DOCUMENT_9',  'FLAG_DOCUMENT_10',  'FLAG_DOCUMENT_11',  'FLAG_DOCUMENT_12',  'FLAG_DOCUMENT_13',  'FLAG_DOCUMENT_14',  'FLAG_DOCUMENT_15',  'FLAG_DOCUMENT_16',  'FLAG_DOCUMENT_17',  'FLAG_DOCUMENT_18',  'FLAG_DOCUMENT_19',  'FLAG_DOCUMENT_20',  'FLAG_DOCUMENT_21'],axis=1,inplace=True)

application_data.drop(labels=["REG_REGION_NOT_LIVE_REGION","LIVE_REGION_NOT_WORK_REGION","LIVE_CITY_NOT_WORK_CITY"],axis=1,inplace=True)

application_data.drop(labels=['OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE'],axis=1,inplace=True)

numerical_features = [feature for feature in application_data.columns if application_data[feature].dtypes != 'O']

DAYS_feature = [feature for feature in numerical_features if 'DAYS' in feature]

application_data["Age"] = application_data.DAYS_BIRTH//(-365) 
application_data[["Age"]].head()
application_data["YEAR_EMPLOYED"] = application_data.DAYS_EMPLOYED//(-365) 
application_data[["YEAR_EMPLOYED"]].describe()
application_data[["YEAR_EMPLOYED"]].value_counts()
application_data["YEAR_EMPLOYED"].replace([-1001],[1],inplace=True)
application_data[["YEAR_EMPLOYED"]].value_counts()
application_data["YEAR_REGISTRATION"] = application_data.DAYS_REGISTRATION//(-365) 
application_data[["YEAR_REGISTRATION"]].describe()
application_data["YEAR_ID_PUBLISH"] = application_data.DAYS_ID_PUBLISH//(-365) 
application_data[["YEAR_ID_PUBLISH"]].describe()
application_data["YEAR_LAST_PHONE_CHANGE"] = application_data.DAYS_LAST_PHONE_CHANGE//(-365) 
application_data[["YEAR_LAST_PHONE_CHANGE"]].describe()
application_data.drop(labels=DAYS_feature, inplace=True, axis=1)

application_data['CNT_FAM_MEMBERS']=application_data['CNT_FAM_MEMBERS'].fillna(application_data['CNT_FAM_MEMBERS'].mode()[0])
application_data['CNT_FAM_MEMBERS'] = application_data['CNT_FAM_MEMBERS'].astype(int)

application_data.drop(labels='HOUR_APPR_PROCESS_START', inplace=True, axis=1)

application_data.YEAR_LAST_PHONE_CHANGE.replace(-0.0, 0.0, inplace=True)

prev_ap_merged = pd.merge(application_data[['SK_ID_CURR', 'TARGET']], previous_application, how='left', on=['SK_ID_CURR'])


prev_ap_merged.drop(['SK_ID_PREV'], axis=1, inplace=True)
prev_ap_merged['NFLAG_LAST_APPL_IN_DAY'] = prev_ap_merged['NFLAG_LAST_APPL_IN_DAY'].fillna(prev_ap_merged['NFLAG_LAST_APPL_IN_DAY'].mode()[0])
prev_ap_merged['NFLAG_INSURED_ON_APPROVAL'] = prev_ap_merged['NFLAG_INSURED_ON_APPROVAL'].fillna(prev_ap_merged['NFLAG_INSURED_ON_APPROVAL'].mode()[0])
for i in ['AMT_ANNUITY',
 'AMT_APPLICATION',
 'AMT_CREDIT',
 'AMT_DOWN_PAYMENT',
 'AMT_GOODS_PRICE']:
  prev_ap_merged[i].fillna(prev_ap_merged[i].median(),inplace = True)
for i in ['AMT_ANNUITY',
 'AMT_APPLICATION',
 'AMT_CREDIT',
 'AMT_DOWN_PAYMENT',
 'AMT_GOODS_PRICE']:
  prev_ap_merged = prev_ap_merged[prev_ap_merged[i] < np.nanpercentile(prev_ap_merged[i], 99)]
prev_ap_merged.drop(labels='HOUR_APPR_PROCESS_START', inplace=True, axis=1)
prev_ap_merged.drop(['DAYS_FIRST_DRAWING'], axis=1, inplace=True)
prev_ap_merged["YEAR_DECISION"] = prev_ap_merged.DAYS_DECISION//(-365) 
prev_ap_merged["YEAR_FIRST_DUE"] = prev_ap_merged.DAYS_FIRST_DUE//(-365) 
prev_ap_merged["YEAR_LAST_DUE_1ST_VERSION"] = prev_ap_merged.DAYS_LAST_DUE_1ST_VERSION//(-365)
prev_ap_merged["YEAR_LAST_DUE"] = prev_ap_merged.DAYS_LAST_DUE//(-365) 
prev_ap_merged["YEAR_TERMINATION"] = prev_ap_merged.DAYS_TERMINATION//(-365)


for i in ['YEAR_DECISION',
 'YEAR_FIRST_DUE',
 'YEAR_LAST_DUE_1ST_VERSION',
 'YEAR_LAST_DUE',
 'YEAR_TERMINATION']:
  prev_ap_merged[i].fillna(prev_ap_merged[i].median(),inplace = True)
prev_ap_merged['YEAR_LAST_DUE_1ST_VERSION']=prev_ap_merged['YEAR_LAST_DUE_1ST_VERSION'].abs()
prev_ap_merged['SELLERPLACE_AREA'].fillna(prev_ap_merged['SELLERPLACE_AREA'].median(),inplace = True)
prev_ap_merged = prev_ap_merged[prev_ap_merged['SELLERPLACE_AREA'] < np.nanpercentile(prev_ap_merged['SELLERPLACE_AREA'], 99)]
prev_ap_merged['CNT_PAYMENT'].fillna(prev_ap_merged['CNT_PAYMENT'].median(),inplace = True)
prev_ap_merged = prev_ap_merged[prev_ap_merged['CNT_PAYMENT'] < np.nanpercentile(prev_ap_merged['CNT_PAYMENT'], 99)]
prev_ap_merged['RATE_DOWN_PAYMENT'].fillna(prev_ap_merged['RATE_DOWN_PAYMENT'].median(),inplace = True)
prev_ap_merged = prev_ap_merged[prev_ap_merged['RATE_DOWN_PAYMENT'] < np.nanpercentile(prev_ap_merged['RATE_DOWN_PAYMENT'], 99)]
prev_ap_merged['NAME_CONTRACT_TYPE'].fillna(prev_ap_merged.NAME_CONTRACT_TYPE.mode()[0], inplace=True)
prev_ap_merged['WEEKDAY_APPR_PROCESS_START'].fillna(prev_ap_merged.WEEKDAY_APPR_PROCESS_START.mode()[0], inplace=True)
prev_ap_merged.drop(labels=['FLAG_LAST_APPL_PER_CONTRACT'],axis=1,inplace=True)
prev_ap_merged['NAME_CASH_LOAN_PURPOSE'].fillna(prev_ap_merged.NAME_CASH_LOAN_PURPOSE.mode()[0], inplace=True)
prev_ap_merged['NAME_CONTRACT_STATUS'].fillna(prev_ap_merged.NAME_CONTRACT_STATUS.mode()[0], inplace=True)
prev_ap_merged['NAME_PAYMENT_TYPE'].fillna(prev_ap_merged.NAME_PAYMENT_TYPE.mode()[0], inplace=True)
prev_ap_merged['CODE_REJECT_REASON'].fillna(prev_ap_merged.CODE_REJECT_REASON.mode()[0], inplace=True)
prev_ap_merged['NAME_TYPE_SUITE'].fillna(prev_ap_merged.NAME_TYPE_SUITE.mode()[0], inplace=True)
prev_ap_merged['NAME_CLIENT_TYPE'].fillna(prev_ap_merged.NAME_CLIENT_TYPE.mode()[0], inplace=True)
prev_ap_merged['NAME_GOODS_CATEGORY'].fillna(prev_ap_merged.NAME_GOODS_CATEGORY.mode()[0], inplace=True)
prev_ap_merged['NAME_PORTFOLIO'].fillna(prev_ap_merged.NAME_PORTFOLIO.mode()[0], inplace=True)
prev_ap_merged['NAME_PRODUCT_TYPE'].fillna(prev_ap_merged.NAME_PRODUCT_TYPE.mode()[0], inplace=True)
prev_ap_merged['CHANNEL_TYPE'].fillna(prev_ap_merged.CHANNEL_TYPE.mode()[0], inplace=True)
prev_ap_merged['NAME_SELLER_INDUSTRY'].fillna(prev_ap_merged.NAME_SELLER_INDUSTRY.mode()[0], inplace=True)
prev_ap_merged['NAME_YIELD_GROUP'].fillna(prev_ap_merged.NAME_YIELD_GROUP.mode()[0], inplace=True)
prev_ap_merged['PRODUCT_COMBINATION'].fillna(prev_ap_merged.PRODUCT_COMBINATION.mode()[0], inplace=True)

prev_ap_merged.drop(labels='TARGET', axis=1, inplace=True)

loan_process_df = pd.merge(application_data, prev_ap_merged, how='inner', on='SK_ID_CURR')

loan_process_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1287645 entries, 0 to 1287644
Data columns (total 78 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   SK_ID_CURR                    1287645 non-null  int64  
 1   TARGET                        1287645 non-null  int64  
 2   NAME_CONTRACT_TYPE_x          1287645 non-null  object 
 3   CODE_GENDER                   1287645 non-null  object 
 4   FLAG_OWN_CAR                  1287645 non-null  object 
 5   FLAG_OWN_REALTY               1287645 non-null  object 
 6   CNT_CHILDREN                  1287645 non-null  int64  
 7   AMT_INCOME_TOTAL              1287645 non-null  float64
 8   AMT_CREDIT_x                  1287645 non-null  float64
 9   AMT_ANNUITY_x                 1287558 non-null  float64
 10  AMT_GOODS_PRICE_x             1286561 non-null  float64
 11  NAME_TYPE_SUITE_x             1284005 non-null  object 
 12  NAME_INCOME_TYPE            

In [ ]:
#@title #Кодирование численных и категориальных фичесов
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics._classification import accuracy_score, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE


Categorical_features=[feature for feature in loan_process_df.columns if loan_process_df[feature].dtype=='O']
print("\n Категориальные features:: ")
print(Categorical_features)
continous_feature=[feature for feature in loan_process_df.columns if feature not in Categorical_features and feature not in ['SK_ID_CURR','TARGET']]
print("\n Численные features:: ")
print(continous_feature)


from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False,handle_unknown='ignore')
ohe.fit(loan_process_df[Categorical_features])
data_cat=ohe.transform(loan_process_df[Categorical_features])
cat_cols_ohe = list(ohe.get_feature_names(input_features=Categorical_features))
data_cat_final = pd.DataFrame(data_cat, columns = cat_cols_ohe)


from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(loan_process_df[continous_feature])
data_Num=scaler.transform(loan_process_df[continous_feature])


data_Num_final=pd.DataFrame(data_Num, columns=continous_feature)
data = pd.concat([loan_process_df[['SK_ID_CURR','TARGET']].reset_index(drop=True),data_Num_final],axis=1)
data_final = pd.concat([data_cat_final,data], axis = 1)

#Тут Нужно очищать память или сохр. данные в файл т.к. Colab крашится!
import gc
del(application_data)
del(previous_application)
del(numerical_features)
del(DAYS_feature)
del(prev_ap_merged)
del(loan_process_df)
del(Categorical_features)
del(continous_feature)
del(ohe)
del(data_cat)
del(cat_cols_ohe)
del(data_cat_final)
del(scaler)
del(data_Num)
del(data_Num_final)
del(data)
gc.collect()


data_final.drop(labels='SK_ID_CURR',axis=1,inplace=True)


#Сплит данных
y = data_final['TARGET'].values
X = data_final.drop(['TARGET'], axis=1)

X = X.fillna(0)
np.nan_to_num(y, copy=False, nan = 0.0)
#y = y.fillna(0)

del(data_final)
gc.collect()



 Категориальные features:: 
['NAME_CONTRACT_TYPE_x', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE_x', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START_x', 'ORGANIZATION_TYPE', 'NAME_CONTRACT_TYPE_y', 'WEEKDAY_APPR_PROCESS_START_y', 'NAME_CASH_LOAN_PURPOSE', 'NAME_CONTRACT_STATUS', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_TYPE_SUITE_y', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE', 'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION']

 Численные features:: 
['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT_x', 'AMT_ANNUITY_x', 'AMT_GOODS_PRICE_x', 'REGION_POPULATION_RELATIVE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'REG_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CI

0

In [ ]:
#@title #3. Сделайте снижение размерности (SVD, PCA, t-SNE) данных.
#Загружаем предобработанные данные из файла:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y)

del(X)
del(y)
gc.collect()

X_train.info()
#Обучим модель и посмотрим результаты (до снижения размерности)
log_reg=LogisticRegression(penalty='l2',C=0.0001,n_jobs=-1)
log_reg.fit(X_train,y_train) #pca_train,pca_test
log_reg.score(X_test,y_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, log_reg.predict(X_test)))



#Выполним снижение размерности данных
from sklearn.metrics import recall_score,roc_auc_score,f1_score,auc,precision_score
from sklearn.decomposition import PCA
pca=PCA(n_components=150)
pca_train=pca.fit_transform(X_train)
pca_test=pca.fit_transform(X_test)



<class 'pandas.core.frame.DataFrame'>
Int64Index: 862722 entries, 515313 to 24824
Columns: 315 entries, NAME_CONTRACT_TYPE_x_Cash loans to YEAR_TERMINATION
dtypes: float64(315)
memory usage: 2.0 GB
              precision    recall  f1-score   support

           0       0.91      1.00      0.95    387910
           1       0.18      0.00      0.01     37013

    accuracy                           0.91    424923
   macro avg       0.55      0.50      0.48    424923
weighted avg       0.85      0.91      0.87    424923



In [ ]:
#@title #4. Обучите модель классификации. Выведите метрики.
#Обучим новую модель на данных после снижения их размерности, чтобы сравнить
log_reg=LogisticRegression(penalty='l2',C=0.0001,n_jobs=-1)
log_reg.fit(pca_train,y_train) #pca_train,pca_test
log_reg.score(pca_test,y_test)


#Выведем метрики
from sklearn.metrics import classification_report
print(classification_report(y_test, log_reg.predict(pca_test)))


              precision    recall  f1-score   support

           0       0.91      1.00      0.95    387910
           1       0.55      0.01      0.01     37013

    accuracy                           0.91    424923
   macro avg       0.73      0.50      0.48    424923
weighted avg       0.88      0.91      0.87    424923



#5. Сделайте вывод
#Вывод:: 
#Результат показывает, что после применения предобработки и снижения размерности данных - показатель точности улучшается